In [1]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

llm.invoke(prompt)

AIMessage(content='= $4923.38')

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool
from langchain.agents import initialize_agent, AgentType

# OpenAI의 ChatGPT 모델을 사용합니다. temperature는 출력의 다양성을 조절하는 파라미터입니다.
llm = ChatOpenAI(temperature=0.1)

# 두 수를 더하는 함수를 정의합니다.
def plus(a, b):
   return a + b

# Agent를 초기화합니다.
agent = initialize_agent(
   # 사용할 Language Model (LLM)을 지정합니다.
   llm=llm,
   # 자세한 출력을 활성화합니다.
   verbose=True,
   # 사용할 Agent의 유형을 지정합니다. 여기서는 Zero-Shot React 방식의 Structured Chat Agent를 사용합니다.
   agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
   # Agent가 사용할 도구(Tool)을 정의합니다.
   tools=[
       # plus 함수를 기반으로 "Sum Calculator"라는 이름의 Tool을 생성합니다.
       StructuredTool.from_function(
           func=plus,
           name="Sum Calculator",
           # Tool에 대한 설명을 제공합니다. 이 Tool은 두 개의 숫자를 인자로 받아 더한 결과를 반환합니다.
           description="Use this to perform sums of two numbers. This tool take two arguments, both should be numbers. till finish.",
       ),
   ],
)

# Agent에게 입력할 프롬프트를 정의합니다.
prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

# 프롬프트를 Agent에 전달하여 실행합니다.
agent.invoke(prompt)



> Entering new AgentExecutor chain...
Thought: I can use the Sum Calculator tool to find the total cost of all the individual amounts provided.

Action:
```
{
  "action": "Sum Calculator",
  "action_input": {"a": 355.39, "b": 924.87}
}
```  

Observation: 1280.26
Thought:I have calculated the sum of the first two amounts. Now, I will continue to calculate the sum of the remaining amounts to find the total cost.
Action:
```
{
  "action": "Sum Calculator",
  "action_input": {"a": 1280.26, "b": 721.2}
}
```  
Observation: 2001.46
Thought:I have calculated the sum of the first two amounts and then added the third amount. Now, I will continue to calculate the sum of the remaining amounts to find the total cost. 

Action:
```
{
  "action": "Sum Calculator",
  "action_input": {"a": 2001.46, "b": 1940.29}
}
```  
Observation: 3941.75
Thought:I have calculated the sum of the first three amounts and added the fourth amount. Now, I will continue to calculate the sum of the remaining amounts to 

{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': 'The total cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5273.38.'}

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


def plus(inputs):
    a, b = inputs.split(",")
    return float(a) + float(b)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    tools=[
        Tool.from_function(
            func=plus,
            name="Sum Calculator",
            description="Use this to perform sums of two numbers. Use this tool by sending a pair of number separated by a comma.\nExample:1,2",
        ),
    ],
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)



> Entering new AgentExecutor chain...
I need to add up all the given numbers to find the total cost.
Action: Sum Calculator
Action Input: 355.39, 924.87
Observation: 1280.26
Thought:I need to continue adding the remaining numbers to the total.
Action: Sum Calculator
Action Input: 1280.26, 721.2
Observation: 2001.46
Thought:I need to continue adding the remaining numbers to the total.
Action: Sum Calculator
Action Input: 2001.46, 1940.29
Observation: 3941.75
Thought:I need to continue adding the remaining numbers to the total.
Action: Sum Calculator
Action Input: 3941.75, 573.63
Observation: 4515.38
Thought:I need to continue adding the remaining numbers to the total.
Action: Sum Calculator
Action Input: 4515.38, 65.72
Observation: 4581.1
Thought:I need to continue adding the remaining numbers to the total.
Action: Sum Calculator
Action Input: 4581.1, 35.00
Observation: 4616.1
Thought:I need to continue adding the remaining numbers to the total.
Action: Sum Calculator
Action Input: 46

{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': '$5273.38'}

In [4]:

from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


class CalculatorToolArgsSchema(BaseModel):
    a: float = Field(description="The first number")
    b: float = Field(description="The second number")


class CalculatorTool(BaseTool):
    name = "CalculatorTool"
    description = """
    Use this to perform sums of two numbers.
    The first and second arguments should be numbers.
    Only receives two arguments.
    """
    args_schema: Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

    def _run(self, a, b):
        return a + b


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        CalculatorTool(),
    ],
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `CalculatorTool` with `{'a': 355.39, 'b': 924.87}`


1280.26
Invoking: `CalculatorTool` with `{'a': 1280.26, 'b': 721.2}`


2001.46

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..



Invoking: `CalculatorTool` with `{'a': 2001.46, 'b': 1940.29}`


3941.75
Invoking: `CalculatorTool` with `{'a': 3941.75, 'b': 573.63}`


4515.38
Invoking: `CalculatorTool` with `{'a': 4515.38, 'b': 65.72}`


4581.1
Invoking: `CalculatorTool` with `{'a': 4581.1, 'b': 35.0}`


4616.1
Invoking: `CalculatorTool` with `{'a': 4616.1, 'b': 552.0}`


5168.1

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..



Invoking: `CalculatorTool` with `{'a': 5168.1, 'b': 76.16}`


5244.26
Invoking: `CalculatorTool` with `{'a': 5244.26, 'b': 29.12}`


5273.38The total cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5273.38.

> Finished chain.


{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': 'The total cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5273.38.'}

In [1]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.tools import DuckDuckGoSearchResults


llm = ChatOpenAI(temperature=0.1)


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )


class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        StockMarketSymbolSearchToolArgsSchema
    ] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchResults()
        return ddg.run(query)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
    ],
)

prompt = "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Cloudflare stock market symbol'}`


[snippet: At Cloudflare, Inc., we promise to treat your data with respect and will not share your information with any third party. You can unsubscribe to any of the investor alerts you are subscribed to by visiting the 'unsubscribe' section below. ... End of Day Stock Quote: Sales Enterprise Sales Become a Partner Contact Sales: +1 (888) 99 FLARE ..., title: Cloudflare, Inc. - Stock Quote & Chart, link: https://cloudflare.net/stock-information/Stock-Quote--Chart/default.aspx], [snippet: Stock analysis for Cloudflare Inc (NET:New York) including stock price, stock chart, company news, key statistics, fundamentals and company profile., title: NET: Cloudflare Inc Stock Price Quote - New York - Bloomberg, link: https://www.bloomberg.com/quote/NET:US], [snippet: Real-time Cloudflare stock price (NYSE: NET) plus other valuable data points like day range, year

{'input': "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have.",
 'output': "The stock market symbol for Cloudflare is NET.\n\nNow, let's analyze Cloudflare's stock to see if it's a potential good investment. Let me gather some information on Cloudflare's stock performance and financials.\nI will now gather information on Cloudflare's stock performance and financials to help analyze if it's a potential good investment."}

In [3]:
import os
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchResults

llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-16k")
alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")

class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for. Example query: Stock Market Symbol for Apple Company")

class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    """
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchResults()
        return ddg.run(query)

class CompanyOverviewArgsSchema(BaseModel):
    symbol: str = Field(description="Stock symbol of the company. Example: AAPL, TSLA")

class CompanyOverviewTool(BaseTool):
    name = "CompanyOverview"
    description = """
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()

class CompanyIncomeStatementTool(BaseTool):
    name = "CompanyIncomeStatement"
    description = """
    Use this to get the income statement of a company for the last 3 years.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()["annualReports"][:3]  # Limit to the last 3 years

class CompanyStockPerformanceTool(BaseTool):
    name = "CompanyStockPerformance"
    description = """
    Use this to get the weekly performance of a company stock for the last 6 months.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={alpha_vantage_api_key}")
        data = r.json()["Weekly Time Series"]
        return {k: data[k] for k in list(data.keys())[:26]}  # Limit to the last 6 months (26 weeks)

agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyStockPerformanceTool(),
    ],
)

prompt = "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Stock Market Symbol for Cloudflare'}`


Stock analysis for Cloudflare Inc (NET:New York) including stock price, stock chart, company news, key statistics, fundamentals and company profile. About NET. Cloudflare, Inc. operates as a cloud services provider that delivers a range of services to businesses worldwide. The company provides an integrated cloud-based security solution to secure a range of combination of platforms, including public cloud, private cloud, on-premise, software-as-a-service applications, and IoT devices; and website and application security products ... Real-time Cloudflare stock price (NYSE: NET) ... Cloudflare Enters Observability Market with Acquisition to Enhance Serverless Performance. Business Wire Apr 5, 2024 3:33pm. At Cloudflare, Inc., we promise to treat your data with respect and will not share your information with any third party. You can unsubscribe to any

{'input': "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment.",
 'output': "Here is the financial information for Cloudflare Inc (NET):\n\n- Stock Symbol: NET\n- Description: CloudFlare, Inc. operates a cloud platform that offers a range of network services to companies around the world.\n- Market Capitalization: $32,752,925,000\n- Revenue (TTM): $1,296,745,000\n- Gross Profit (TTM): $742,631,000\n- Net Income (TTM): -$183,949,000\n- EBITDA (TTM): -$151,990,000\n- EPS (TTM): -$0.55\n- Profit Margin (TTM): -14.2%\n- Operating Margin (TTM): -11.8%\n- Return on Assets (TTM): -4.34%\n- Return on Equity (TTM): -26.5%\n- Quarterly Revenue Growth (YoY): 32%\n- Analyst Target Price: $100.9\n- 52-Week High: $116\n- 52-Week Low: $39.9\n\nBased on the financial information provided, Cloudflare Inc has shown strong revenue growth and a positive gross profit. However, the compa

In [6]:
from langchain.agents import create_sql_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

llm = ChatOpenAI(
    temperature=0.1,
    # model_name="gpt-4-0613",
)
db = SQLDatabase.from_uri("sqlite:///movies.sqlite")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)

agent.invoke(
    "Give me the movies that have the highest votes but the lowest budgets and give me the name of their directors also include their gross revenue."
)



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


directors, movies
Invoking: `sql_db_schema` with `movies`



CREATE TABLE movies (
	id INTEGER NOT NULL, 
	original_title TEXT, 
	budget TEXT, 
	popularity REAL, 
	director_id INTEGER, 
	gross_revenue REAL, 
	PRIMARY KEY (id)
)

/*
3 rows from movies table:
id	original_title	budget	popularity	director_id	gross_revenue
1	Avatar	237000000	150.437577	1	2787965087.0
2	Pirates of the Caribbean: At World's End	300000000	139.082615	2	961000000.0
3	Spectre	245000000	107.376788	3	880674609.0
*/
Invoking: `sql_db_query` with `SELECT original_title, directors.name AS director, gross_revenue FROM movies JOIN directors ON movies.director_id = directors.id ORDER BY votes DESC, budget ASC LIMIT 10`


Error: (sqlite3.OperationalError) no such column: votes
[SQL: SELECT original_title, directors.name AS director, gross_revenue FROM movies JOIN directors ON movies.director_id = directors.id ORDER BY votes DESC, budget ASC 

{'input': 'Give me the movies that have the highest votes but the lowest budgets and give me the name of their directors also include their gross revenue.',
 'output': "The movies with the highest votes but the lowest budgets, along with their directors and gross revenue, are:\n\n1. Movie: Avatar, Director: James Cameron, Gross Revenue: 2787965087.0\n2. Movie: Pirates of the Caribbean: At World's End, Director: Gore Verbinski, Gross Revenue: 961000000.0\n3. Movie: The Dark Knight Rises, Director: Christopher Nolan, Gross Revenue: 1084939099.0\n4. Movie: Spectre, Director: Sam Mendes, Gross Revenue: 880674609.0\n5. Movie: John Carter, Director: Andrew Stanton, Gross Revenue: 284139100.0"}